In [ ]:
# 로지스틱 회귀 : 선형회귀 + 활성함수 == 확률 
# 이진분류  BCE 손실함수

In [1]:
import numpy as np
import pandas as pd

In [2]:
from copy import deepcopy
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_breast_cancer

In [4]:
cencer = load_breast_cancer()
df =pd.DataFrame(cencer.data, columns=cencer.feature_names)
df['class'] = cencer.target

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [7]:
data = torch.FloatTensor(df.values)
x = data[:,:-1]
y = data[:,-1:]
x.shape, y.shape

(torch.Size([569, 30]), torch.Size([569, 1]))

In [8]:
# 학습,검증,테스트  6 2 2
int(data.size(0)*.6), int(data.size(0)*.2)

(341, 113)

In [15]:
# shuffle
from tables import index
# from random import sample
# sample(range(data.size(0)),data.size(0) )
indices = torch.randperm(data.size(0))
x_data =  torch.index_select(x,dim = 0,index = indices )
y_data =  torch.index_select(y,dim = 0,index = indices )

In [25]:
# 정규화
x_scaled = StandardScaler().fit_transform(x_data)

x_scaled = [
  x_data[:341],
  x_data[341:341+113], 
  x_data[341+113:] 
]

y_scaled = [
  y_data[:341],
  y_data[341:341+113], 
  y_data[341+113:] 
]

In [26]:
# linear - lekaryrelu  ..... linear - sigmoid
#  optimizer
#  loss
# 하이퍼 파라메터 설정 - for 훈련(배치개념)

In [28]:
model = nn.Sequential(    
      nn.Linear(30,25),
      nn.LeakyReLU(),
      nn.Linear(25,20),
      nn.LeakyReLU(),
      nn.Linear(20,15),
      nn.LeakyReLU(),
      nn.Linear(15,5),
      nn.LeakyReLU(),
      nn.Linear(5,1),
      nn.Sigmoid()    
)
optimizer = optim.Adam(model.parameters())

In [29]:
n_epochs = 10000
batch_size = 32
interval = 100
ealy_stop = 100

In [31]:
lowest_loss = np.inf
best_model = None
lowest_epoch = np.inf

In [32]:
x_scaled[0].size(0)

torch.Size([341, 30])

In [ ]:
train_hist, valid_hist = [],[]
for epoch in range(n_epochs):
  indices = torch.randperm(x_scaled[0].size(0))  # 341
  x_ = torch.index_select(x_scaled[0], dim = 0 )
  y_ = torch.index_select(y_scaled[0], dim = 0 )

  x_ = x_.split(batch_size,dim = 0)
  y_ = y_.split(batch_size,dim = 0)

  train_loss, valid_loss = 0,0
  y_hat = []

  for x_i, y_i in zip(x_,y_):
    y_hat_i = model(x_i)
    loss = F.binary_cross_entropy(y_hat_i,y_i) # 순서가 있음 

    optimizer.zero_grad()

    loss.backward()
    optimizer.step()
    train_loss += float(loss)  # memory leak 방지를 위해서 
  train_loss = train_loss / len(x_)

  

